# Support Vector Regression

Magdalena Claessen (579362)

Die Support Vector Machine (SVM) gehört zum überwachten Lernen des maschinellen Lernens. Das Ziel des Algorithmus ist, Daten zu klassifizieren, indem es eine Hyperebene mit maximalen Abstand zwischen den Klassen in einem n-dimensionalen Raum findet. Demnach eignet sich die Support Vector Machine für Klassifizierungsprobleme. Aufgrund dessen, dass ein Regressionsproblem in Form einer Zeitreihe vorliegt, kann die SVM nicht verwendet werden. Stattdessen eignet sich die Support Vector Regression (SVR). Die SSVR sucht eine Funktion, die die Datenpunkte innerhalb eines vogegebenen Toleranzbereichs (ε) möglichst gut approximiert. Datenpunkte, die innerhalb des ε-Toleranzbereichs liegen, werden nicht bestraft, während Punkte außerhalb eine Strafe basierend auf der Entfernung zur vorhergesagten Funktion erhalten. 

https://link.springer.com/chapter/10.1007/978-3-030-89010-0_9#Sec10

Für die Auswahl der maschinellen Lernverfahrens wird das Paper "Support Vector Regression (SVR) Model for Seasonal Time Series Data" von H. Muthiah, U. Sa'adah und A. Efendi herangezogen. In diesem Paper wird die Zeitreihe der Stromlasten in der Region Ost-Java (Indonesien) mit insgesamt 182 Datenpunkten mit der SVR trainiert und vorhergesagt. Dabei wird die Performance der verschiedenen Kernels linear und rbf im Vergleich getestet. Das Ergebnis der Vorhersage nach Aufbereitung mit SARIMA (Seasonal Autoregressive Integrated Moving Average), dem Training und der Modellevaluation mit dem Root Mean Squared Error (RMSE) ist, dass der lineare Kernel besser Vorhersagen für diese Zeitreihe trifft. 

In diesem Notebook werden zunächst die bereits im Notebook "6_preprocessing" beschriebenden Schritte zur Datenvorverarbeitung durchgeführt. Anschließend folgt die Implementierung des finalen Modells. Daruffolgend ist die Herleitung dieses Modell inklusive zusätzlichem Feature Engineering und Hyperparameter-Tuning beschrieben. Für die Implementierung der Support Vector Regression für den Rossmann Store Datensatz wird die Python-Bibliothek Scikit-learn verwendet. Diese Bibliothek ist speziell für das maschinelle Lernen konzipiert und stellt die SVR zur Verfügung.  

https://scikit-learn.org/stable/modules/svm.html

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html 


SARIMAX Modell vorher verwenden – Vorteil Saisonalität und Trends

https://d1wqtxts1xzle7.cloudfront.net/104852295/Tran_Le_Chau_109-libre.pdf?1691506461=&response-content-disposition=inline%3B+filename%3DSupport_Vector_Regression_based_on_Grid.pdf&Expires=1726780529&Signature=FUMNCz6TGP1hP29NjOQgaD5hOAccW~cJ5hv5YbLUotibquZBcehEewilyn~wcgRLs-MVdx1LW2psh0NqyxvWZw2ZFMoqVg8V0XqR1yZfQfQF-tukhhpO7odn9u4wdfmVQUjgdJEa1YLpUctkdV8PLn~JXNQSGOda6~pl5YlXIiCR8c0TTMCwQxaouWI10yT~tb~5xkaT4wGE1-kj8JHP2pUSw4b-N5XuzN8mn~a0qFlfO8W~UT~XuOcFPcXQV4GEUC8MuXdWIP8HUN08X76dlAkePMGAnrm6ZNP6owat7AhkkzGc2mWm3x4n8cgIFbLeeoZWPnFVMm0L8C6qPfpQIA__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA


In [28]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectFromModel
import pandas as pd
import numpy as np
from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector

# Metrik

In [29]:
# Angepasste RMSPE-Funktion, die Tage mit 0 Sales ignoriert
def rmspe(y_true, y_pred):
    # Nur Fälle berücksichtigen, bei denen y_true nicht 0 ist
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

In [30]:
# Zeitreihen-Kreuzvalidierung einrichten
tscv = TimeSeriesSplit(n_splits=5)

# Daten einlesen und Split

In [31]:
# Laden des Datensatzes
data_cleaned = "../data/cleaned_train_marie.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Definiere die numerischen und kategorischen Features
#numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']
#numerical_features = ['year', 'month', 'day', 'week_of_year', 'fourier_sin_365', 'fourier_cos_365'] #Verwendet bei Pipeline mit Standardwerten
#numerical_features = ['year', 'month', 'day', 'week_of_year', 'fourier_sin_365', 'fourier_cos_365', 'lag_1', 'lag_7'] # Verwendet für Feature Engineering mit Lag Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'fourier_sin_365', 'fourier_cos_365', 'days_since_last_holiday',	'days_until_next_holiday'] # für weitere Features
#numerical_features = ['year', 'month', 'day', 'week_of_year', 'fourier_sin_365', 'fourier_cos_365', 'days_since_last_holiday',	'days_until_next_holiday', 'is_weekend', 'is_school_holiday', 'is_holiday', 'day_of_year', 'day_of_week'] #Feature Selektion

# Bereits encodierte Features
already_encoded_features = ['Promo', 'promo2']

# Noch nicht encodierte kategorische Features
categorical_features_to_encode = ['DayOfWeek', 'StoreType', 'StateHoliday','Assortment', 'Store']

In [32]:
data = data.sort_values('Date')

data = data[data ['Open']!=0]
data = data[data ['Sales']>0]

# # Stichprobengröße von 20%
# sample_fraction = 0.2 
# sample_size = int(len(data) * sample_fraction)
# data_sample = data.iloc[:sample_size]

# Zielvariable und Features definieren
X = data.drop(['Sales', 'Date'], axis=1)
# X = data.drop(['Sales', 'Date', 'Open', 'day_of_week', 'day_of_year', 'is_weekend', 'is_holiday', 'is_school_holiday', 'lag_1', 'lag_7'], axis=1) # Verwendet bei Pipeline mit Standardwerten
#X = data.drop(['Sales', 'Date', 'Open', 'day_of_week', 'day_of_year', 'is_weekend', 'is_holiday', 'is_school_holiday'], axis=1)
y = data['Sales']

# Berechnung der Anzahl der Testdaten (20 % des Datensatzes)
test_size = int(len(data) * 0.2)

# Aufteilen der Daten in Trainings- und Testdaten
X_train, X_test = X.iloc[:-test_size], X.iloc[-test_size:]
y_train, y_test = y.iloc[:-test_size], y.iloc[-test_size:]

In [33]:
X_train.head()

,Store,DayOfWeek,Open,Promo,StateHoliday,year,month,day,week_of_year,day_of_week,...,is_school_holiday,StoreType,Assortment,promo2,fourier_sin_365,fourier_cos_365,days_since_last_holiday,days_until_next_holiday,lag_1,lag_7
511718,562,2,1,0,a,2013,1,1,1,1,...,1,b,c,0,0.017213,0.999852,0,0.0,20212.0,18462.0
699720,769,2,1,0,a,2013,1,1,1,1,...,1,b,b,1,0.017213,0.999852,0,0.0,11188.0,10125.5
237398,262,2,1,0,a,2013,1,1,1,1,...,1,b,a,0,0.017213,0.999852,0,0.0,20224.5,18799.5
466458,512,2,1,0,a,2013,1,1,1,1,...,1,b,b,0,0.017213,0.999852,0,0.0,5420.0,5160.0
863186,948,2,1,0,a,2013,1,1,1,1,...,1,b,b,0,0.017213,0.999852,0,0.0,6790.0,6496.5


# Preprocessor

In [34]:
#Erstelle den Preprocessor für numerische und kategorische Features (ohne Datumsextraktion)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Skalierung für numerische und bereits encodierte Features
        ('enc', 'passthrough', already_encoded_features),  # Bereits encodierte Features durchschleusen (keine weitere Transformation)
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)  # Nur noch nicht encodierte Features encodieren
    ])

# Finales Modell LinearSVR

Text
{'model__loss': 'epsilon_insensitive', 'model__epsilon': 0.21544346900318845, 'model__C': 46.41588833612773}
= 0.31919044322710344

'model__C': 10, 'model__epsilon': 0.1, 'model__loss': 'epsilon_insensitive'} = 0.3311307497458499

In [39]:
linear_svr = LinearSVR(loss='epsilon_insensitive', epsilon=0.21544346900318845, C=46.41588833612773, random_state=42)

linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', linear_svr)])

In [40]:
linear_rmspe_scores = []

for fold, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {fold + 1}")

    # Train- und Testdaten für diesen Fold
    X_train_fold = X_train.iloc[train_index]
    X_test_fold = X_train.iloc[test_index]
    y_train_fold = y_train.iloc[train_index]
    y_test_fold = y_train.iloc[test_index]

    linear_pipeline.fit(X_train_fold, y_train_fold)

    y_pred_fold = linear_pipeline.predict(X_test_fold)

    linear_rmspe_score = rmspe(y_test_fold, y_pred_fold)
    linear_rmspe_scores.append(linear_rmspe_score)

    print(f"RMSPE für Fold {fold + 1}: {linear_rmspe_score}")

linear_mean_rmspe = np.mean(linear_rmspe_scores)
print(f"Durchschnittlicher RMSPE über alle Folds: {linear_mean_rmspe}")

Fold 1
RMSPE für Fold 1: 0.35698227551788597
Fold 2
RMSPE für Fold 2: 0.3018909231536752
Fold 3
RMSPE für Fold 3: 0.2832838768203259
Fold 4
RMSPE für Fold 4: 0.2507451876463854
Fold 5
RMSPE für Fold 5: 0.46275148559097673
Durchschnittlicher RMSPE über alle Folds: 0.3311307497458499


# Pipeline mit Standardwerten

Um ein effektives Modell für die Support Vector Regression zu entwickeln, wird eine Pipeline implementiert, die den Preprocessor und das Modell umfasst. Die Pipeline automatisiert den Prozess der SVR und vereinfacht den Umgang mit Daten und Modell. (https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) Die verwendete Support Vector Machine von Scikit-Learn bietet verschiedene Parameter zur Anpassung des Modells:
- Kernel: Dieser Parameter gibt den verwendeten Kerneltyp an. Mögliche Werte sind linear, rbf, poly, sigmoid, precomuted (default: rbf)
- C: Der Regulierungsparameter, der je nach Größe die Daten bestraft. Bei einem niedrigen Wert werden mehr Fehler erlaubt, wobei bei einem hohen C-Wert weniger Fehler erlaubt sind. (default: 1.0)
- Epsilon: Gibt die Toleranz gegenüber Fehlern an. Innerhalb des Toleranzbereichs werden die Punkte nicht bestraft. (default: 0.1)
- Gamma: dieser Parameter wird angegeben, wenn der Kernel rbf, poly oder sigmoid ist. Mögliche Werte sind scale oder auto. (default: scale)


Zunächst wird die SVR mit den dafault-Werten auf den Trainingsdaten trainiert. Die hierfür verwendeten Features sind:
- numerische Features: 'year', 'month', 'day', 'week_of_year', 'fourier_sin_365', 'fourier_cos_365'
- encoded Features: 'Promo', 'promo2'
- kategorische Features: 'DayOfWeek', 'StoreType', 'StateHoliday','Assortment', 'Store'

Der erste Durchlauf mit den default-Werten dient dazu, die Grundleistung des Modells ohne Optimierung zu überprüfen. Die Ergebnisse der fünfachen Time Series Kreuzvalidierung zeigen folgenden RMSPE Werte:
- Fold 1: 0.5392254908405987
- Fold 2: 0.4809429942883717
- Fold 3: 0.4518889103767877
- Fold 4: 0.45680753303355526
- Fold 5: -

Nach über 10 Stunden wurde der Durchlauf gestoppt und der durchschnittlicher RMSPE wurde manuell über die vier berechneten Folds ermittelt. Der durchschnittliche RMSPE ist 0.4822162321348283.

Aufgrund dieser schlechten RMSPE-Werte wird die LinearSVR als Vergleich herangezogen. Der Unterschied zur SVR liegt in der Verwendung eines linearen Kernels, der es ermöglicht, das Modell effizienter auf größere Datenmengen zu trainieren. Ein Hyperparameter-Tuning, das mit 20% des Datensatzes durchgeführt wurde, zeigt, dass der lineare Kernel die besten Ergebnisse liefert. Die lineare SVR von Scikit-Learn bietet folgende Parameter zur Anpassung des Modells:
- Epsilon: Gibt die Toleranz gegenüber Fehlern an. Innerhalb des Toleranzbereichs werden die Punkte nicht bestraft. (default: 0.0)
- C: Der Regulierungsparameter, der je nach Größe die Daten bestraft. Ein niedriger C-Wert erlaubt mehr Fehler und führt zu einer stärkeren Regularisierung, was ein einfacheres Modell ergibt. Ein hoher C-Wert bestraft Fehler stärker und lässt weniger Fehler zu. Das Modell wir komplexer. (default: 1.0)
- loss: Die Verlustfunktion, die für das Modell verwendet wird. (default: epsilon_insensitive)

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVR.html 

Die RMSPE-Werte der fünfachen Time Serie Kreuzvalidierung bei oben genannten Features sind folgende:
- Fold 1: 0.45677323476966736
- Fold 2: 0.39130488386280415
- Fold 3: 0.394032395948371
- Fold 4: 0.3747369458031788
- Fold 5: 0.5392642148781003

Zur Vergleichbarkeit mit der SVR wurde hier ebenfalls der durchschnittliche RMSPE manuell nach vier Folds berechnet. Der durchschnittlicher RMSPE nach vier Folds beträgt 0.4042118650960053, während der nach fünf Folds 0.4312223350524243 ist. Diese Ergebnisse verdeutlichen, dass der Fehler bei der linearen SVR geringer ist und aus diesem Grund wird der lineare Ansatz weiter verfolgt.

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVR.html#sklearn.svm.LinearSVR 

Warum Support Vector Machine und warum Linear besser ist als RBF: https://ieomsociety.org/proceedings/2021indonesia/574.pdf 

## Support Vector Regression (SVR)

In [28]:
svr = SVR()

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', svr)])

In [29]:
rsmpe_scores = []

for fold, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {fold + 1}")

    # Train- und Testdaten für diesen Fold
    X_train_fold = X_train.iloc[train_index]
    X_test_fold = X_train.iloc[test_index]
    y_train_fold = y_train.iloc[train_index]
    y_test_fold = y_train.iloc[test_index]

    pipeline.fit(X_train_fold, y_train_fold)

    y_pred_fold = pipeline.predict(X_test_fold)

    rmspe_score = rmspe(y_test_fold, y_pred_fold)
    rsmpe_scores.append(rmspe_score)

    print(f"RMSPE für Fold {fold + 1}: {rmspe_score}")

Fold 1
RMSPE für Fold 1: 0.5392254908405987
Fold 2
RMSPE für Fold 2: 0.4809429942883717
Fold 3
RMSPE für Fold 3: 0.4518889103767877
Fold 4
RMSPE für Fold 4: 0.45680753303355526
Fold 5


## Pipline mit GridSearchCV für SVR bei 20% der Datenmenge

In [32]:
svr = SVR()

In [33]:
# Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', svr)])

In [34]:
param_grid = {
    'model__C': [0.1, 1, 10, 100],
    'model__epsilon': [0.01, 0.1, 0.5, 1],
    'model__kernel': ['linear', 'rbf']
}

grid_search = GridSearchCV(pipeline, param_grid, cv=tscv, scoring=rmspe_scorer, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['year',
                                                                          'month',
                                                                          'day',
                                                                          'week_of_year',
                                                                          'fourier_sin_365',
                                                                          'fourier_cos_365']),
                                                                        ('enc',
                                                                         'passthrough',
                                                                         ['Promo',
                                                                          'promo2']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['DayOfWeek',
                                                                          'StoreType',
                                                                          'StateHoliday',
                                                                          'Assortment',
                                                                          'Store'])])),
                                       ('model', SVR())]),
             n_jobs=-1,
             param_grid={'model__C': [0.1, 1, 10, 100],
                         'model__epsilon': [0.01, 0.1, 0.5, 1],
                         'model__kernel': ['linear', 'rbf']},
             scoring=make_scorer(rmspe, greater_is_better=False, response_method='predict'))

In [35]:
print("Beste Parameter: ", grid_search.best_params_)

Beste Parameter:  {'model__C': 100, 'model__epsilon': 1, 'model__kernel': 'linear'}


In [36]:
# Vorhersage
y_pred = grid_search.predict(X_test)

rmspe_score = rmspe(y_test, y_pred)
print(f'RMSPE auf Testdaten: {rmspe_score}')

RMSPE auf Testdaten: 0.4829542601858057


## Linear Support Vector Regression (LinearSVR)

In [11]:
linear_svr = LinearSVR()

linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', linear_svr)])

In [14]:
linear_rmspe_scores = []

for fold, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {fold + 1}")

    # Train- und Testdaten für diesen Fold
    X_train_fold = X_train.iloc[train_index]
    X_test_fold = X_train.iloc[test_index]
    y_train_fold = y_train.iloc[train_index]
    y_test_fold = y_train.iloc[test_index]

    linear_pipeline.fit(X_train_fold, y_train_fold)

    y_pred_fold = linear_pipeline.predict(X_test_fold)

    linear_rmspe_score = rmspe(y_test_fold, y_pred_fold)
    linear_rmspe_scores.append(linear_rmspe_score)

    print(f"RMSPE für Fold {fold + 1}: {linear_rmspe_score}")

linear_mean_rmspe = np.mean(linear_rmspe_scores)
print(f"Durchschnittlicher RMSPE über alle Folds: {linear_mean_rmspe}")

Fold 1
RMSPE für Fold 1: 0.4565170808222471
Fold 2
RMSPE für Fold 2: 0.390202120954233
Fold 3
RMSPE für Fold 3: 0.39484612063419683
Fold 4
RMSPE für Fold 4: 0.37475680784698767
Fold 5
RMSPE für Fold 5: 0.5393270490612893
Durchschnittlicher RMSPE über alle Folds: 0.4311298358637908


LinearSVR 20%
Fold 1
RMSPE für Fold 1: 0.5637879994942535
Fold 2
RMSPE für Fold 2: 0.5251295677027914
Fold 3
RMSPE für Fold 3: 0.4355646328212453
Fold 4
RMSPE für Fold 4: 0.5736173302639209
Fold 5
RMSPE für Fold 5: 0.4280862071764491

Durchschnittlicher RMSPE über alle Folds: 0.5050140062864357

Linear alle Daten
Fold 1
RMSPE für Fold 1: 0.45677323476966736
Fold 2
RMSPE für Fold 2: 0.39130488386280415
Fold 3
RMSPE für Fold 3: 0.394032395948371
Fold 4
RMSPE für Fold 4: 0.3747369458031788
Fold 5
RMSPE für Fold 5: 0.5392642148781003

Durchschnittlicher RMSPE über alle Folds: 0.4312223350524243

SVM Standard 20%
Fold 1
RMSPE für Fold 1: 0.6716704351689681
Fold 2
RMSPE für Fold 2: 0.5538782009893275
Fold 3
RMSPE für Fold 3: 0.4976484948391306
Fold 4
RMSPE für Fold 4: 0.5640210048259803
Fold 5
RMSPE für Fold 5: 0.4921612546849901

Durchschnittlicher RMSPE über alle Folds: 0.5558758781016794

# Feature Engineering LinearSVR

## Lag Features

Für die weitere Verbesserung des Modells werden die Lag-Features 1 und 7 hinzugefügt und dynamisch aktualisiert. Jedoch hat sich der Fehler von 43.11% auf 63.95% verschlechtert, sodass die Lag-Features wieder vernachlässigt werden. 

In [25]:
linear_svr = LinearSVR()

linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', linear_svr)])

In [32]:
#Funktion zur Vorhersage der nächsten x Tage (wie im Originalcode)
def predict_multiple_days_with_updates(X_last, test_data, num_days):
    predictions = []
    X_next = X_last.copy()
    X_next = pd.DataFrame([X_next])  # In DataFrame umwandeln
    last_lags = list(X_last[['lag_1']].values.flatten())  # Initialisiere letzte Lags

    for day in range(num_days):
        # Preprocess die Daten vor der Vorhersage
        X_next_preprocessed = linear_pipeline.named_steps['preprocessor'].transform(X_next)
        
        # Vorhersage des nächsten Tages
        y_next_pred = linear_pipeline.named_steps['model'].predict(X_next_preprocessed)[0]
        predictions.append(y_next_pred)

        # Aktualisiere die Lag-Features basierend auf der Vorhersage
        last_lags.append(y_next_pred)
        last_lags.pop(0)  # Aktualisiere Lag-Werte

        X_next.loc[:, 'lag_1'] = y_next_pred
        X_next.loc[:, 'lag_7'] = last_lags[0]

        # Aktualisiere andere Features (Tag, Monat, Woche)
        X_next.loc[:, 'day'] += 1
        if X_next.loc[:, 'day'].values[0] > 31:
            X_next.loc[:, 'day'] = 1
            X_next.loc[:, 'month'] += 1
        if X_next.loc[:, 'month'].values[0] > 12:
            X_next.loc[:, 'month'] = 1
            X_next.loc[:, 'year'] += 1

        X_next.loc[:, 'week_of_year'] = (X_next['week_of_year'] % 52) + 1
        X_next.loc[:, 'DayOfWeek'] = (X_next['DayOfWeek'] % 7) + 1

        # Promo- und andere Features aus Testdaten übernehmen
        if day < len(test_data):
            X_next.loc[:, 'Promo'] = test_data.iloc[day]['Promo']
            X_next.loc[:, 'Open'] = test_data.iloc[day]['Open']
            X_next.loc[:, 'StateHoliday'] = test_data.iloc[day]['StateHoliday']

    return predictions

In [33]:
linear_rmspe_scores = []

for fold, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {fold + 1}")

    # Train- und Testdaten für diesen Fold
    X_train_fold = X_train.iloc[train_index]
    X_test_fold = X_train.iloc[test_index]
    y_train_fold = y_train.iloc[train_index]
    y_test_fold = y_train.iloc[test_index]

    linear_pipeline.fit(X_train_fold, y_train_fold)

    x_last = X_train_fold.iloc[-1]
    num_days = len(X_test_fold)

    predictions = predict_multiple_days_with_updates(x_last, X_test_fold, num_days)

    linear_rmspe_score = rmspe(y_test_fold, y_pred_fold)
    linear_rmspe_scores.append(linear_rmspe_score)

    print(f"RMSPE für Fold {fold + 1}: {linear_rmspe_score}")

linear_mean_rmspe = np.mean(linear_rmspe_scores)
print(f"Durchschnittlicher RMSPE über alle Folds: {linear_mean_rmspe}")

Fold 1
RMSPE für Fold 1: 0.6840230357143419
Fold 2
RMSPE für Fold 2: 0.6627651179884798
Fold 3
RMSPE für Fold 3: 0.6487884631457884
Fold 4
RMSPE für Fold 4: 0.6250614898158731
Fold 5
RMSPE für Fold 5: 0.5770603562678673
Durchschnittlicher RMSPE über alle Folds: 0.63953969258647


## Weitere Features

mit days_since_last_holiday, days_until_next_holiday

In [114]:
linear_svr = LinearSVR(random_state=42)

linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', linear_svr)])

In [104]:
linear_rmspe_scores = []

for fold, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {fold + 1}")

    # Train- und Testdaten für diesen Fold
    X_train_fold = X_train.iloc[train_index]
    X_test_fold = X_train.iloc[test_index]
    y_train_fold = y_train.iloc[train_index]
    y_test_fold = y_train.iloc[test_index]

    linear_pipeline.fit(X_train_fold, y_train_fold)

    y_pred_fold = linear_pipeline.predict(X_test_fold)

    linear_rmspe_score = rmspe(y_test_fold, y_pred_fold)
    linear_rmspe_scores.append(linear_rmspe_score)

    print(f"RMSPE für Fold {fold + 1}: {linear_rmspe_score}")

linear_mean_rmspe = np.mean(linear_rmspe_scores)
print(f"Durchschnittlicher RMSPE über alle Folds: {linear_mean_rmspe}")

Fold 1
RMSPE für Fold 1: 0.4456277961767345
Fold 2
RMSPE für Fold 2: 0.3860227106148182
Fold 3
RMSPE für Fold 3: 0.3988297296419189
Fold 4
RMSPE für Fold 4: 0.3744126796727805
Fold 5
RMSPE für Fold 5: 0.546141619319136
Durchschnittlicher RMSPE über alle Folds: 0.43020690708507753


Marie Features mit meinem generierten Datensatz 2:
Durchschnittlicher RMSPE über alle Folds: 0.3132433655713715

## Forward Selection

verschiedene Anzahl an Features ausprobieren und dann visualisieren

In [9]:
linear_svr = LinearSVR(random_state=42)

In [10]:
feature_selector = SequentialFeatureSelector(
    linear_svr, 
    n_features_to_select=5, 
    direction='forward', 
    scoring=rmspe_scorer, 
    cv=tscv,
    n_jobs=1
    )

In [11]:
linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', feature_selector),
    ('model', linear_svr)])

In [12]:
linear_rmspe_scores = cross_val_score(linear_pipeline, X_train, y_train, scoring=rmspe_scorer, cv=tscv)

# Berechne den Durchschnittlichen RMSPE über alle Folds
linear_mean_rmspe = np.mean(linear_rmspe_scores)
print(f"Durchschnittlicher RMSPE über alle Folds: {linear_mean_rmspe}")

linear_pipeline.fit(X_train, y_train)

# Extrahiere die ausgewählten Features
selected_features = linear_pipeline.named_steps['feature_selection'].get_support()

print(f"Ausgewählte Features: {X_train.columns[selected_features]}")

: 

In [21]:
# ALT
linear_rmspe_scores = []
selected_features_list = []

for fold, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {fold + 1}")

    # Train- und Testdaten für diesen Fold
    X_train_fold = X_train.iloc[train_index]
    X_test_fold = X_train.iloc[test_index]
    y_train_fold = y_train.iloc[train_index]
    y_test_fold = y_train.iloc[test_index]

    linear_pipeline.fit(X_train_fold, y_train_fold)

    selected_features = linear_pipeline.named_steps['feature_selection'].get_support()
    selected_features_list.append(X_train.columns[selected_features])

    y_pred_fold = linear_pipeline.predict(X_test_fold)

    linear_rmspe_score = rmspe(y_test_fold, y_pred_fold)
    linear_rmspe_scores.append(linear_rmspe_score)

    print(f"RMSPE für Fold {fold + 1}: {linear_rmspe_score}")
    print(f"Ausgewählte Features für Fold {fold + 1}: {X_train.columns[selected_features]}")

linear_mean_rmspe = np.mean(linear_rmspe_scores)
print(f"Durchschnittlicher RMSPE über alle Folds: {linear_mean_rmspe}")
print(f"Alle ausgewählten Features über die Folds hinweg: {selected_features_list}")

Fold 1


KeyboardInterrupt: 

# Hyperparameter Tuning

Bevor das Hyperparamter Tuning durchgeführt wird, wurde sich in der Gruppe auf die folgenden Features geeinigt, sodass diese für weitere Analysen verwendet werden. 
- numerische Features: 'year', 'month', 'day', 'week_of_year', 'fourier_sin_365', 'fourier_cos_365', 'days_since_last_holiday',	'days_until_next_holiday'
- encoded Features: 'Promo', 'promo2'
- kategorische Features: 'DayOfWeek', 'StoreType', 'StateHoliday','Assortment', 'Store'

Für das Hyperparameter Tuning werden die Methoden RandomizedSearchCV und GridSearchCV gegeneinander getestet, sodass die besten Parameter gefunden werden können. RandomizedSearchCV ist eine kreuzvalidierte Suche, die nicht alle Parameterwerte ausprobiert, sondern mit einer festen Anzahl. Die Anzahl kann durch n_iter angegeben und verändert werden. Im Gegensatz dazu ist GridSearchCV eine kreuzvalidierte Rastersuche, die alle Parameterwerte ausprobiert. Beide Verfahren wurden auf das Modell mit den oben gegnannten Features angwendet und im Anschluss jeweils eine Vorhersage auf den Testdaten gestartet. Die Ergbnisse sind:
- RandomizedSearchCV
    - Beste Parameter: {'model__loss': 'epsilon_insensitive', 'model__epsilon': 0.21544346900318845, 'model__C': 46.41588833612773}
    - RMSPE auf den Testdaten: 0.2773890240861983
- GridSearchCV
    - Beste Parameter: {'model__C': 10, 'model__epsilon': 0.1, 'model__loss': 'epsilon_insensitive'}
    - RMSPE auf den Testdaten: 0.28647300992303587

Demnach sind die Ergebnisse der RandomizedSearchCV besser als die der GridSearchCV, da der RMSPE der Testdaten geringer ist. Die Parameter der RandomizedSearchCV werden für das Modell verwendet.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html


## RandomizedSearchCV

- Fold 1
  - RMSPE für Fold 1: 0.41195277999491664
  - Beste Parameter für Fold: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(2.782559402207126), 'model__C': np.float64(0.021544346900318832)}
- Fold 2
  - RMSPE für Fold 2: 0.6027160270022384
  - Beste Parameter für Fold: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(0.7742636826811278), 'model__C': np.float64(215.44346900318823)}
- Fold 3
  - RMSPE für Fold 3: 0.2714170557481643
  - Beste Parameter für Fold: {'model__loss': 'squared_epsilon_insensitive', 'model__epsilon': np.float64(0.001291549665014884), 'model__C': np.float64(0.021544346900318832)}
- Fold 4
  - RMSPE für Fold 4: 0.23472070548522392
  - Beste Parameter für Fold: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(0.016681005372000592), 'model__C': np.float64(46.41588833612773)}
- Fold 5
  - RMSPE für Fold 5: 0.45408722838055
  - Beste Parameter für Fold: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(0.016681005372000592), 'model__C': np.float64(46.41588833612773)}

- Durchschnittlicher RMSPE über alle Folds: 0.3949787593222186

- Beste Parameter insgesamt: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(0.21544346900318845), 'model__C': np.float64(46.41588833612773)}

- RMSPE auf den Testdaten: 0.2773890240861983

In [18]:
linear_svr = LinearSVR(random_state=42)

linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', linear_svr)])

In [19]:
linear_rmspe_scores = []
param_distributions = {
    'model__C': np.logspace(-3, 3, 10),  # Logarithmische Verteilung der Werte für C
    'model__epsilon': np.logspace(-4, 1, 10),
    'model__loss': ['epsilon_insensitive', 'squared_epsilon_insensitive']
}

for fold, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {fold + 1}")

    # Train- und Testdaten für diesen Fold
    X_train_fold = X_train.iloc[train_index]
    X_test_fold = X_train.iloc[test_index]
    y_train_fold = y_train.iloc[train_index]
    y_test_fold = y_train.iloc[test_index]

    # Hyperparameteroptimierung für diesen Fold
    random_search = RandomizedSearchCV(linear_pipeline, param_distributions, n_iter=50, cv=5, scoring=rmspe_scorer, random_state=42)
    random_search.fit(X_train_fold, y_train_fold)

    # Bestes Modell mit den optimierten Hyperparametern
    best_model = random_search.best_estimator_

    # Vorhersage für den Testfold
    y_pred_fold = best_model.predict(X_test_fold)

    # Berechnung des RMSPE für diesen Fold
    linear_rmspe_score = rmspe(y_test_fold, y_pred_fold)
    linear_rmspe_scores.append(linear_rmspe_score)

    print(f"RMSPE für Fold {fold + 1}: {linear_rmspe_score}")
    print("Beste Parameter für Fold:", random_search.best_params_)

# Durchschnittlichen RMSPE über alle Folds berechnen
linear_mean_rmspe = np.mean(linear_rmspe_scores)
print(f"Durchschnittlicher RMSPE über alle Folds: {linear_mean_rmspe}")

Fold 1


c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warni

RMSPE für Fold 1: 0.41195277999491664
Beste Parameter für Fold: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(2.782559402207126), 'model__C': np.float64(0.021544346900318832)}
Fold 2


c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RMSPE für Fold 2: 0.6027160270022384
Beste Parameter für Fold: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(0.7742636826811278), 'model__C': np.float64(215.44346900318823)}
Fold 3
RMSPE für Fold 3: 0.2714170557481643
Beste Parameter für Fold: {'model__loss': 'squared_epsilon_insensitive', 'model__epsilon': np.float64(0.001291549665014884), 'model__C': np.float64(0.021544346900318832)}
Fold 4


c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RMSPE für Fold 4: 0.23472070548522392
Beste Parameter für Fold: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(0.016681005372000592), 'model__C': np.float64(46.41588833612773)}
Fold 5
RMSPE für Fold 5: 0.45408722838055
Beste Parameter für Fold: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(0.016681005372000592), 'model__C': np.float64(46.41588833612773)}
Durchschnittlicher RMSPE über alle Folds: 0.3949787593222186


In [20]:
final_random_search = RandomizedSearchCV(linear_pipeline, param_distributions, n_iter=50, cv=tscv, scoring=rmspe_scorer, random_state=42)
final_random_search.fit(X_train, y_train)

print("Beste Parameter insgesamt:", final_random_search.best_params_)

c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\schmu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Beste Parameter insgesamt: {'model__loss': 'epsilon_insensitive', 'model__epsilon': np.float64(0.21544346900318845), 'model__C': np.float64(46.41588833612773)}


In [21]:
# Finale Modellanpassung mit den besten Parametern
final_model = final_random_search.best_estimator_
final_model.fit(X_train, y_train)

# Vorhersage auf Testdaten
y_test_pred = final_model.predict(X_test)

# Berechnung des RMSPE auf den Testdaten
final_rmspe_score = rmspe(y_test, y_test_pred)
print(f"RMSPE auf den Testdaten: {final_rmspe_score}")

RMSPE auf den Testdaten: 0.2773890240861983


## GridSearchCV

In [24]:
linear_svr = LinearSVR(random_state=42)

linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', linear_svr)])

In [26]:
param_grid = {
    'model__C': [0.01, 0.1, 1, 10, 100],
    'model__epsilon': [0.001, 0.01, 0.1, 1],
    'model__loss': ['epsilon_insensitive', 'squared_epsilon_insensitive']
}

grid_search = GridSearchCV(linear_pipeline, param_grid, cv=tscv, scoring=rmspe_scorer)
grid_search.fit(X_train, y_train)

print("Beste Parameter:", grid_search.best_params_)

Beste Parameter: {'model__C': 10, 'model__epsilon': 0.1, 'model__loss': 'epsilon_insensitive'}


In [27]:
# Vorhersagen mit dem besten Modell
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Berechnung des RMSPE auf den Testdaten
rmspe_error = rmspe(y_test, y_pred)
print(f"RMSPE auf den Testdaten: {rmspe_error}")

RMSPE auf den Testdaten: 0.28647300992303587


# Visualisierung

# Vorhersage

# Fazit

Text

# Pipeline mit RandomizedSearch und Feature Selector

In [48]:
svr = SVR()

feature_selector = SequentialFeatureSelector(
    estimator=LinearRegression(),  # Ersetze durch dein Modell
    n_features_to_select='auto',   # Anzahl der Features kann auch manuell gesetzt werden
    direction='forward',            # Vorwärtsselektion
    cv=tscv                        # Cross-Validation
)

# 2. Erstelle die Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Dein Preprocessing-Schritt
    ('feature_selector', feature_selector),  # Feature-Auswahl
    ('model', svr)  # Dein Modell
])

In [49]:
# Definiere die Parameterverteilung
param_dist = {
    'feature_selector__n_features_to_select': [5, 10, 13], 
    'model__C': [0.1, 1, 10, 100, 500, 1000],
    'model__epsilon': [0.01, 0.1, 0.5, 1, 2],
    'model__kernel': ['linear', 'rbf'],
    'model__gamma': ['scale', 'auto']
}

# RandomizedSearch anwenden
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=50, cv=tscv, scoring=rmspe_scorer, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)


KeyboardInterrupt: 

In [ ]:
# Beste Parameter ausgeben
print("Beste Parameter: ", random_search.best_params_)
print("Beste Features: ", random_search.best_estimator_.named_steps['feature_selector'].get_support(indices=True))

1. Modell einmal zum laufen bekommen
2. Hyperparameter optimieren --> GridSearchCV
3. mehr Features hinzufügen
4. Feature Engineering
5. ganzes Modell trainieren - keine Stichprobe mehr
6. für 6 Wochen die Vorhersage machen

Stichprobe 0.1: Beste Parameter:  {'model__C': 0.1, 'model__epsilon': 0.5, 'model__kernel': 'rbf'}, RMSPE auf Testdaten: 0.5016386546722168
Stichprobe 0.2: Beste Parameter:  {'model__C': 10, 'model__epsilon': 0.5, 'model__kernel': 'linear'}, RMSPE auf Testdaten: 0.5316766360768732
Stichprobe 0.2 mit mehreren Features: Beste Parameter:  {'model__C': 100, 'model__epsilon': 1, 'model__kernel': 'linear'}, RMSPE auf Testdaten: 0.4829542601858057
